In [2]:
import numpy as np
import cv2
import keras
import os
from threading import Thread
from gtts import gTTS 
from playsound import playsound 
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pyttsx3
import mediapipe as mp
import time
import os
from gtts import gTTS 
from playsound import playsound 
import cv2
import re
import time
from moviepy.editor import *
import keyboard
import speech_recognition as sr
import time


background= None
def say_text(text):
    engine = pyttsx3.init()
    while engine._inLoop:
        pass
    engine.setProperty("rate", 125)
    engine.say(text)
    engine.runAndWait()

def cal_accum_avg(frame, accumulated_weight):
    global background
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

def segment_hand(frame, threshold=25):
    global background
    global contour
    diff = cv2.absdiff(background.astype("uint8"), frame)
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)    
    #Fetching contours in the frame (These contours can be of hand or any other object in foreground) ...
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # If length of contours list = 0, means we didn't get any contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand 
        hand_segment_max_cont = max(contours, key=cv2.contourArea)  
        contour = max(contours, key = cv2.contourArea)
        # Returning the hand segment(max contour) and the thresholded image of hand...
        return (thresholded, hand_segment_max_cont)

def sign_to_speech():
    model = keras.models.load_model("best_model.h5")
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    data=""
    text=""
    count_same_frame = 0
    count_same_frame1 = 0
    count_same_frame2 = 0
    background = None
    accumulated_weight = 0.5
    ROI_top = 100
    ROI_bottom = 300
    ROI_right = 150
    ROI_left = 350
    word_dict = {0:'0', 1:'1', 2:'10', 3:'2', 4:'3', 5:'4', 6:'5', 7:'6', 8:'7', 9:'8', 10:'9', 11:'A', 12:'B', 13:'C', 14:'E',
             15:'F', 16:'G', 17:'H', 18:'I', 19:'L', 20:'Y', 21:'Yes', 22:'are', 23:'hello', 24:'how',
             25:'is', 26:'my', 27:'name', 28:'no', 29:'thanks for listening', 30:'you'}

    cam = cv2.VideoCapture(0)
    num_frames =0
    with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
        while True:
            ret, frame = cam.read()
            frame = cv2.flip(frame, 1)
            image1 = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
        #image1 = cv2.flip(image1, 1)
            image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
            image1.flags.writeable = False
            results = hands.process(image1)
            image1.flags.writeable = True
            image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image1, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.imshow('MediaPipe Hands', image1)
            frame_copy = frame.copy()
            roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
            gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
            if num_frames < 151:
                cal_accum_avg(gray_frame, accumulated_weight)
                cv2.putText(frame_copy, str(num_frames)+"For", (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                cv2.putText(frame_copy, "Saving Backgroung. Please wait for 150 frames", (80, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)
                cv2.putText(frame_copy, "Keep background for better prediction", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)
         
            else: 
                hand = segment_hand(gray_frame)
                if hand is not None:
                    thresholded, hand_segment = hand
                    cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
                    cv2.imshow("Thesholded Hand Image", thresholded)
                    thresholded = cv2.resize(thresholded, (64, 64))
                    thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
                    thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
                    pred = model.predict(thresholded)
                    cv2.putText(frame_copy, word_dict[np.argmax(pred)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                    old_text=text
                    cv2.putText(frame_copy, str(count_same_frame), (30, 20), cv2.FONT_ITALIC, 0.5, (255,0,0), 1)
                    if cv2.contourArea(contour) > 4000:
                        text = word_dict[np.argmax(pred)]
                        if old_text == text:
                            count_same_frame += 1
                        if old_text!=text:
                            count_same_frame=0
                        if count_same_frame == 30:
                            data = data + text
                            if data.startswith('I/Me '):
                                data = data.replace('I/Me ', 'I ')
                            elif data.endswith('I/Me '):
                                data = data.replace('I/Me ', 'me ')
                            data+=" "
                            count_same_frame =0
                    elif count_same_frame < 20:
                        data+="."
                        if data != '':
                            Thread(target=say_text, args=(data, )).start()
                        text = ""
                        data = ""
                        count_same_frame=0
                else:
                    if data != '':
                        Thread(target=say_text, args=(data, )).start()
                    text = ""
                    data = ""
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
            cv2.putText(blackboard, " ", (180, 50), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (255, 0,0))
            cv2.putText(blackboard, "Predicted text- " + text, (30, 100), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (255, 255, 0))
            cv2.putText(blackboard, data, (10, 240), cv2.FONT_HERSHEY_TRIPLEX, 0.65, (255, 255, 255))
            cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)
            num_frames += 1
            cv2.putText(frame_copy, str(count_same_frame), (30, 20), cv2.FONT_ITALIC, 0.5, (0,255,255), 1)
            res = np.hstack((frame_copy, blackboard))
            cv2.imshow("Recognizing gesture", res)
            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                break

        cam.release()
        cv2.destroyAllWindows()
    while True:
        if keyboard.is_pressed('q'):  # if key 'q' is pressed 
            text_to_sign()
            break
        if keyboard.is_pressed('w'):  # if key 'w' is pressed 
            speech_to_sign()
            break
        if keyboard.is_pressed('e'):  # if key 'e' is pressed 
            sign_to_speech()
            break
        if keyboard.is_pressed('t'):  # if key 't' is pressed 
            text_to_speech()
            break
        if keyboard.is_pressed('r'):
            print("program terminated")# if key 'r' is pressed 
            break

def text_to_sign():
    t=input("Enter your message: ")
    img=re.split('\s+', t)
    for k in range(len(img)):
        img[k]= img[k]+".jpg"
    print(img)
    
    while len(img)==1:
        res = cv2.imread('D:/data/code/'+str(t)+'.jpg')
        cv2.imshow("image", res)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    cv2.destroyAllWindows()
    if len(img)>1:
        time.sleep(5)
        clips = [ImageClip(m).set_duration(3)
                  for m in img]
        concat_clip = concatenate_videoclips(clips)
        concat_clip.preview()
    while True:
        if keyboard.is_pressed('q'):  # if key 'q' is pressed 
            text_to_sign()
            break
        if keyboard.is_pressed('w'):  # if key 'w' is pressed 
            speech_to_sign()
            break
        if keyboard.is_pressed('e'):  # if key 'e' is pressed 
            sign_to_speech()
            break
        if keyboard.is_pressed('t'):  # if key 't' is pressed 
            text_to_speech()
            break
        if keyboard.is_pressed('r'):
            print("program terminated")# if key 'r' is pressed 
            break
        
def speech_to_sign():
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as audio_file:
        print("Speak Please")
        r.adjust_for_ambient_noise(audio_file)
        audio = r.listen(audio_file)
        t= r.recognize_google(audio)
        print("Converting Speech to Text...")
        print("You said: " + t)
    img=re.split('\s+', t)
    for k in range(len(img)):
        img[k]= img[k]+".jpg"
    print(img)
    
    while len(img)==1:
        res = cv2.imread('D:/data/code/'+str(t)+'.jpg')
        cv2.imshow("image", res)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    if len(img)>1:
        time.sleep(5)
        clips = [ImageClip(m).set_duration(3)
                  for m in img]
        concat_clip = concatenate_videoclips(clips)
        concat_clip.preview()
    while True:
        if keyboard.is_pressed('q'):  # if key 'q' is pressed 
            text_to_sign()
            break
        if keyboard.is_pressed('w'):  # if key 'w' is pressed 
            speech_to_sign()
            break
        if keyboard.is_pressed('e'):  # if key 'e' is pressed 
            sign_to_speech()
            break
        if keyboard.is_pressed('t'):  # if key 't' is pressed 
            text_to_speech()
            break
        if keyboard.is_pressed('r'):
            print("program terminated")# if key 'r' is pressed 
            break

def text_to_speech():
    msg=input("enter the message")
    say_text(msg)
    while True:
        if keyboard.is_pressed('q'):  # if key 'q' is pressed 
            text_to_sign()
            break
        if keyboard.is_pressed('w'):  # if key 'w' is pressed 
            speech_to_sign()
            break
        if keyboard.is_pressed('e'):  # if key 'e' is pressed 
            sign_to_speech()
            break
        if keyboard.is_pressed('t'):  # if key 't' is pressed 
            text_to_speech()
            break
        if keyboard.is_pressed('r'):
            print("program terminated")# if key 'r' is pressed 
            break


while True:
    if keyboard.is_pressed('q'):  # if key 'q' is pressed 
        text_to_sign()
        break
    if keyboard.is_pressed('w'):  # if key 'w' is pressed 
        speech_to_sign()
        break
    if keyboard.is_pressed('e'):  # if key 'e' is pressed 
        sign_to_speech()
        break
    if keyboard.is_pressed('t'):  # if key 't' is pressed 
        text_to_speech()
        break
    if keyboard.is_pressed('r'):
        print("program terminated")# if key 'r' is pressed 
        break


enter the messageI am fine.
program terminated
